In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
from imageio import imread

import warnings
warnings.filterwarnings("ignore")

C:\Users\misak\AppData\Local\Temp\ipykernel_31876\2458891310.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
xdf = pd.read_csv("games.csv")

In [4]:
xdf

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",17K,3.8K,4.6K,4.8K
1,1,Hades,"Dec 10, 2019",['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,21K,3.2K,6.3K,3.6K
2,2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4.3K,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,30K,2.5K,5K,2.6K
3,3,Undertale,"Sep 15, 2015","['tobyfox', '8-4']",4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,28K,679,4.9K,1.8K
4,4,Hollow Knight,"Feb 24, 2017",['Team Cherry'],4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",21K,2.4K,8.3K,2.3K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,1507,Back to the Future: The Game,"Dec 22, 2010",['Telltale Games'],3.2,94,94,"['Adventure', 'Point-and-Click']",Back to the Future: The Game is one of Telltal...,['Very enjoyable game. The story adds onto the...,763,5,223,67
1508,1508,Team Sonic Racing,"May 21, 2019","['Sumo Digital', 'Sega']",2.9,264,264,"['Arcade', 'Racing']",Team Sonic Racing combines the best elements o...,"['jogo morto mas bom', 'not my cup of tea', ""C...",1.5K,49,413,107
1509,1509,Dragon's Dogma,"May 22, 2012",['Capcom'],3.7,210,210,"['Brawler', 'RPG']","Set in a huge open world, Dragon’s Dogma: Dark...","['Underrated.', 'A grandes rasgos, es como un ...",1.1K,45,487,206
1510,1510,Baldur's Gate 3,"Oct 06, 2020",['Larian Studios'],4.1,165,165,"['Adventure', 'RPG', 'Strategy', 'Tactical', '...","An ancient evil has returned to Baldur's Gate,...",['Bu türe bu oyunla girmeye çalışmak hataydı s...,269,79,388,602


In [5]:
xdf.head()

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,"Feb 25, 2022","['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",17K,3.8K,4.6K,4.8K
1,1,Hades,"Dec 10, 2019",['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,21K,3.2K,6.3K,3.6K
2,2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4.3K,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,30K,2.5K,5K,2.6K
3,3,Undertale,"Sep 15, 2015","['tobyfox', '8-4']",4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,28K,679,4.9K,1.8K
4,4,Hollow Knight,"Feb 24, 2017",['Team Cherry'],4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",21K,2.4K,8.3K,2.3K


In [6]:
xdf.shape

(1512, 14)

In [7]:
xdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1512 non-null   int64  
 1   Title              1512 non-null   object 
 2   Release Date       1512 non-null   object 
 3   Team               1511 non-null   object 
 4   Rating             1499 non-null   float64
 5   Times Listed       1512 non-null   object 
 6   Number of Reviews  1512 non-null   object 
 7   Genres             1512 non-null   object 
 8   Summary            1511 non-null   object 
 9   Reviews            1512 non-null   object 
 10  Plays              1512 non-null   object 
 11  Playing            1512 non-null   object 
 12  Backlogs           1512 non-null   object 
 13  Wishlist           1512 non-null   object 
dtypes: float64(1), int64(1), object(12)
memory usage: 165.5+ KB


In [8]:
dataG = xdf[[
      'Title',
      'Release Date',
      'Genres',
      'Summary',
      'Reviews',
      'Rating',
      'Playing',
      'Backlogs',
      'Wishlist',
      'Plays']]
dataG.head()

,Title,Release Date,Genres,Summary,Reviews,Rating,Playing,Backlogs,Wishlist,Plays
0,Elden Ring,"Feb 25, 2022","['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",4.5,3.8K,4.6K,4.8K,17K
1,Hades,"Dec 10, 2019","['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,4.3,3.2K,6.3K,3.6K,21K
2,The Legend of Zelda: Breath of the Wild,"Mar 03, 2017","['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,4.4,2.5K,5K,2.6K,30K
3,Undertale,"Sep 15, 2015","['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,4.2,679,4.9K,1.8K,28K
4,Hollow Knight,"Feb 24, 2017","['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",4.4,2.4K,8.3K,2.3K,21K


In [9]:
dataG.isnull().sum()

Title            0
Release Date     0
Genres           0
Summary          1
Reviews          0
Rating          13
Playing          0
Backlogs         0
Wishlist         0
Plays            0
dtype: int64

In [10]:
dataG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         1512 non-null   object 
 1   Release Date  1512 non-null   object 
 2   Genres        1512 non-null   object 
 3   Summary       1511 non-null   object 
 4   Reviews       1512 non-null   object 
 5   Rating        1499 non-null   float64
 6   Playing       1512 non-null   object 
 7   Backlogs      1512 non-null   object 
 8   Wishlist      1512 non-null   object 
 9   Plays         1512 non-null   object 
dtypes: float64(1), object(9)
memory usage: 118.3+ KB


In [11]:
#removing rows with NULL value
#for example, games with no reviews

In [12]:
#drop all NULL values
dataG=dataG.dropna()

#reset index rows of dataframe to avoid non-consecutive index values
dataG=dataG.reset_index(drop=True)

print(f"Shape of new dataset is: {dataG.shape}")

Shape of new dataset is: (1499, 10)


In [13]:
#there were 13 rows wil null values
#Double checking if there are any null values left in the dataset
dataG.isnull().values.any()


False

In [14]:
#Extract year from release data and drop Release date column
import datetime

def extract_year(date_str):
    try:
        date_object = datetime.datetime.strptime(date_str, "%b %d, %Y")
        return date_object.year
    except ValueError:
        return pd.NA  # Handle potential format errors (optional)

dataG['Year'] = dataG['Release Date'].apply(extract_year)
#dataG.drop('Release Date',axis=1) 
data=dataG.drop('Release Date',axis=1) 
data.head()

,Title,Genres,Summary,Reviews,Rating,Playing,Backlogs,Wishlist,Plays,Year
0,Elden Ring,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",4.5,3.8K,4.6K,4.8K,17K,2022
1,Hades,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,4.3,3.2K,6.3K,3.6K,21K,2019
2,The Legend of Zelda: Breath of the Wild,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,4.4,2.5K,5K,2.6K,30K,2017
3,Undertale,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,4.2,679,4.9K,1.8K,28K,2015
4,Hollow Knight,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",4.4,2.4K,8.3K,2.3K,21K,2017


In [15]:
data.isnull().values.any()

True

In [16]:
#Release date column contain strings with "releases on TBD" which means the game has not been released yet
#extracting the year from this column cause this string value to be null and now we also have to remove the rows containing it
#drop all NULL values
data=data.dropna()

#reset index rows of datframe to avoid non-consecutive index values
data=data.reset_index(drop=True)

#convert date column from string to int
data['Year']=data['Year'].astype(int)
#df['points'] = df['points'].astype(str).astype(int)

print(f"Shape of new dataset is: {data.shape}")


Shape of new dataset is: (1497, 10)


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497 entries, 0 to 1496
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Title     1497 non-null   object 
 1   Genres    1497 non-null   object 
 2   Summary   1497 non-null   object 
 3   Reviews   1497 non-null   object 
 4   Rating    1497 non-null   float64
 5   Playing   1497 non-null   object 
 6   Backlogs  1497 non-null   object 
 7   Wishlist  1497 non-null   object 
 8   Plays     1497 non-null   object 
 9   Year      1497 non-null   int32  
dtypes: float64(1), int32(1), object(8)
memory usage: 111.2+ KB


In [18]:
# Converts the numbers with K to multiple by 1000 and convert into whole integers since K = 1000. Eg. 1K = 1 * 1000
def extract_value(value_str):
    if value_str.upper().endswith('K'):
        value_str = value_str[:-1]
        value_num = float(value_str) * 1000
    else:
        value_num = float(value_str)
    return value_num


In [19]:
games_clean = data.copy()

#replaces K for 1000
games_clean["Backlogs"] = games_clean["Backlogs"].str.replace("K","000")
games_clean["Backlogs"] = games_clean["Backlogs"].str.replace(".","")
games_clean["Wishlist"] = games_clean["Wishlist"].str.replace("K","000")
games_clean["Wishlist"] = games_clean["Wishlist"].str.replace(".","")

games_clean[['Backlogs', 'Wishlist']] = games_clean[['Backlogs', 'Wishlist']].apply(pd.to_numeric)

#combines column backlogs and wishlist
#for people who are interested
games_clean['Interested'] = games_clean["Backlogs"] + games_clean["Wishlist"]
games_clean['Currently Playing']=games_clean['Playing'].apply(extract_value)
games_clean['Total Plays']=games_clean['Plays'].apply(extract_value)

#convert to float (except year) for consistency
games_clean['Currently Playing']=games_clean['Currently Playing'].astype(float)
games_clean['Interested']=games_clean['Interested'].astype(float)
games_clean['Total Plays']=games_clean['Total Plays'].astype(float)

#removes Playing, Backlogs, and Wishlist from df
games_clean.pop("Backlogs")
games_clean.pop("Wishlist")
games_clean.pop("Playing")
games_clean.pop("Plays")


games_clean.head()

,Title,Genres,Summary,Reviews,Rating,Year,Interested,Currently Playing,Total Plays
0,Elden Ring,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",4.5,2022,94000.0,3800.0,17000.0
1,Hades,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,4.3,2019,99000.0,3200.0,21000.0
2,The Legend of Zelda: Breath of the Wild,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,4.4,2017,31000.0,2500.0,30000.0
3,Undertale,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,4.2,2015,67000.0,679.0,28000.0
4,Hollow Knight,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",4.4,2017,106000.0,2400.0,21000.0


In [20]:
# Check if there are any other NULL values
games_clean.info()
games_clean.isnull().values.any()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497 entries, 0 to 1496
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              1497 non-null   object 
 1   Genres             1497 non-null   object 
 2   Summary            1497 non-null   object 
 3   Reviews            1497 non-null   object 
 4   Rating             1497 non-null   float64
 5   Year               1497 non-null   int32  
 6   Interested         1497 non-null   float64
 7   Currently Playing  1497 non-null   float64
 8   Total Plays        1497 non-null   float64
dtypes: float64(4), int32(1), object(4)
memory usage: 99.5+ KB


False

In [21]:
#Remove brackets and ' in Genres. 
#check type if index 0 exist
print (type(games_clean.loc[0, 'Genres']))


#check type generally, index can be `DatetimeIndex`, `FloatIndex`...
print (type(games_clean.loc[games_clean.index[0], 'Genres']))
games_clean['Genres'] = games_clean['Genres'].str.strip('[]')
games_clean['Genres'] = games_clean['Genres'].str.replace(r"' '", " ", regex=True)

games_clean.head()

<class 'str'>
<class 'str'>


,Title,Genres,Summary,Reviews,Rating,Year,Interested,Currently Playing,Total Plays
0,Elden Ring,"'Adventure', 'RPG'","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",4.5,2022,94000.0,3800.0,17000.0
1,Hades,"'Adventure', 'Brawler', 'Indie', 'RPG'",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,4.3,2019,99000.0,3200.0,21000.0
2,The Legend of Zelda: Breath of the Wild,"'Adventure', 'RPG'",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,4.4,2017,31000.0,2500.0,30000.0
3,Undertale,"'Adventure', 'Indie', 'RPG', 'Turn Based Strat...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,4.2,2015,67000.0,679.0,28000.0
4,Hollow Knight,"'Adventure', 'Indie', 'Platform'",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",4.4,2017,106000.0,2400.0,21000.0


In [22]:
games_clean.describe() #not useful, will take out later

,Rating,Year,Interested,Currently Playing,Total Plays
count,1497.000000,1497.000000,1497.000000,1497.000000,1497.000000
mean,3.718236,2012.062124,16187.757515,269.981296,6315.252505
std,0.532018,8.491472,21032.136839,427.783884,5891.900341
min,0.700000,1980.000000,9.000000,0.000000,8.000000
25%,3.400000,2007.000000,715.000000,44.000000,1900.000000
50%,3.800000,2014.000000,2799.000000,115.000000,4300.000000
75%,4.100000,2019.000000,25000.000000,302.000000,9100.000000
max,4.600000,2023.000000,106000.000000,3800.000000,33000.000000


In [23]:
#Check if there are any empty info in the data.
games_clean.isna().sum()

Title                0
Genres               0
Summary              0
Reviews              0
Rating               0
Year                 0
Interested           0
Currently Playing    0
Total Plays          0
dtype: int64

NLP Part

Cleaning text from random unicode characters (In this case is the []), and keeping only letters, numbers, spaces. Applies to summary and reviews

In [24]:
def decode(column):
    column = column.str.decode("unicode_escape")\
    .str.replace(r'[^a-zA-Z1-9\ ]', '', regex=True).str.strip()
    return column

In [25]:
games_clean.loc[:,"Summary"] = decode(games_clean["Summary"])
games_clean.loc[:,"Reviews"] = decode(games_clean["Reviews"])

This part of the code removes ONLY completely non-english reviews, basically games with reviews that does not contain english. Games with reviews with english reviews will still stay.
It detects which reviews are in English. Puts the english reviews in a new dataframe called english_reviews

In [26]:
# Function to detect the language of a review using langid with confidence threshold
def detect_language_with_langdetect(line): 
    from langdetect import detect_langs
    try: 
        langs = detect_langs(line) 
        for item in langs: 
            # The first one returned is usually the one that has the highest probability
            return item.lang, item.prob 
    except: return "err", 0.0 

# Apply language detection to each review and add a 'IsEnglish' column to the DataFrame
games_clean['LanguageDetection'] = games_clean['Reviews'].apply(detect_language_with_langdetect)

#Filter out non-English reviews based on the threshold
english_reviews = games_clean[games_clean['LanguageDetection'].apply(lambda x: x[0] == 'en' and x[1] >= 0.8)]
games_clean = english_reviews

# Output the results with language and confidence percentages
# for idx, (lang, confidence) in enumerate(english_reviews['LanguageDetection']):
#     print(f"Review {idx + 1}: Language={lang}, Confidence={confidence:.2%}")

# print(games_clean)

Why not remove the Spanish Words? 
The sentiment values are still valid. As long as the review contains english words, the sentiment analysis will still use those english words and evaluate whether the reviews are positive or negative. From some games (Eg. 1482, Dragon Ball GT: Final Bout), the sentiment values were below 0, indicating a negative review. Although these reviews contained spanish words, the sentiment values were still correct judging by the english sentences that were in these reviews.

To make the sentiment value more useful, we will find games that are above 4.0 rating containing the top words from the summary of games extracted from positive reviews. This will give us a more accurate result of what are the top and popular/highly rated games the people are looking out for.

In [27]:
print(games_clean.tail(20).to_string())

                                Title                                                             Genres                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

Resets the index after cleaning of random unicode characters is done

In [28]:
games_clean.reset_index(inplace=True, drop=True)
games_clean.head()

,Title,Genres,Summary,Reviews,Rating,Year,Interested,Currently Playing,Total Plays,LanguageDetection
0,Elden Ring,"'Adventure', 'RPG'",Elden Ring is a fantasy action and open world ...,The first playthrough of elden ring is one of ...,4.5,2022,94000.0,3800.0,17000.0,"(en, 0.9999973834970206)"
1,Hades,"'Adventure', 'Brawler', 'Indie', 'RPG'",A roguelite hack and slash dungeon crawler in ...,convinced this is a roguelike for people who d...,4.3,2019,99000.0,3200.0,21000.0,"(en, 0.9999964013724008)"
2,Undertale,"'Adventure', 'Indie', 'RPG', 'Turn Based Strat...",A small child falls into the Underground where...,soundtrack is tied for 1 with nier automata a...,4.2,2015,67000.0,679.0,28000.0,"(en, 0.9999963447914968)"
3,Hollow Knight,"'Adventure', 'Indie', 'Platform'",A 2D metroidvania with an emphasis on close co...,this games worldbuilding is incredible with it...,4.4,2017,106000.0,2400.0,21000.0,"(en, 0.9999965872483545)"
4,Minecraft,"'Adventure', 'Simulator'",Minecraft focuses on allowing the player to ex...,Minecraft is what you make of it Unfortunately...,4.3,2011,11230.0,1800.0,33000.0,"(en, 0.8571408703435166)"


Processes incoming text data using spacy to determine the base form of each word in summary & reviews, then returns a list of lemmatized words.  
Lemmatized words are one single form word.
Eg.  "builds", "building",or "built" to the lemma "build"

In [29]:
nlp = spacy.load('en_core_web_sm')

def lemmatize(text):
    doc = nlp(text)
    tokens = [token for token in doc if not token.is_punct]
    lemmas = [token.lemma_ if token.pos_ != 'PRON' else token.orth_ for token in tokens]
    return lemmas

# games_clean['Reviews'] = games_clean['Reviews'].apply(lemmatize)

Here, we are using TextBlob to indicate whether the reviews in the dataset lean towards positive or negative. 
If a sentiment value is above 0, it means it is a positive review.
If it's below 0, it indicates a negative value.
The range of sentiment value is [-1, 1]. The value that is closer to -1 or 1 indicates stronger positive/negative sentiments.

In [33]:
from textblob import TextBlob

# Function to calculate sentiment polarity using TextBlob
def get_sentiment(review):
    blob = TextBlob(review)
    return blob.sentiment.polarity

# Apply sentiment analysis to each review and add a 'Sentiment' column to the DataFrame
games_clean['Sentiment'] = games_clean['Reviews'].apply(get_sentiment)

# Filter positive reviews based on sentiment polarity
positive_reviews = games_clean[games_clean['Sentiment'] > 0]['Reviews']

Converts text data to TF-IDF vectors.
 
TF-IDF is a measure that "evaluates how relevant a word is to a document in a collection of documents". It increases proportionally to the number of times a word appears in a dataset, but is offset by the number of datasets that contain the word.
Example: Words that are common in every document, such as this, what, and if, rank low even though they may appear many times, since they don’t mean much to that document in particular.

This code creates a matrix with TF-IDF weights for each word. 
Obtains a list of feature names that represent which words are included in these vectors. 

Instead of comparing multiple datasets, will try comparing between reviews and summary to determine the relevancy of the words. From there, can roughly determine which words are relevant and appear very frequently, giving us a gauge of what kind of games people usually prefer and what kind of audience the games generally cater to.

Now let's do it for reviews. Here we are using only positive reviews to get a gauge of what kind of games people usually praise


In [34]:
tfidf_positive = TfidfVectorizer(stop_words="english", tokenizer=lemmatize) #Reduces common words like 'and', 'the' etc as it is not significant.
tfidf_matrix_positive = tfidf_positive.fit_transform(positive_reviews)
feature_names_positive = tfidf_positive.get_feature_names_out()

In [35]:
# Calculate TF-IDF scores for each word in the positive reviews.
tfidf_scores_positive = tfidf_matrix_positive.sum(axis=0)

# Create a DataFrame to display common words and their TF-IDF scores in positive reviews.
common_words_positive = pd.DataFrame(tfidf_scores_positive, columns=feature_names_positive).transpose()
common_words_positive.columns = ['TF-IDF Score']

# Filter out empty strings and whitespace characters.
common_words_positive = common_words_positive[common_words_positive.index.str.strip() != '']

# Filter out specifc number of letters from words. 4 because 1-4 words usually consists of generic words that are redundant.
common_words_positive = common_words_positive[common_words_positive.index.str.len() > 4]

# Sort the DataFrame by TF-IDF score in descending order.
common_words_positive = common_words_positive.sort_values(by='TF-IDF Score', ascending=False)

# Display top common words without blank spaces and single-letter words.
top_common_words_positive = common_words_positive.head(20)
print(top_common_words_positive)

            TF-IDF Score
really         34.823032
story          30.344822
great          27.754245
character      24.801626
gameplay       23.457546
think          21.820185
pretty         20.337987
level          20.012964
world          17.291408
series         16.921028
combat         16.798913
design         16.788157
mario          16.760509
experience     16.308568
thing          14.522023
little         13.834617
enjoy          13.632345
amazing        13.607617
favorite       13.402497
original       12.818759


The code below extracts key words from Summary. The summary selected are from the positive reviews that we have identified previously. This would give a better gauge of what types of games the people are favouring based on the summary of the game description.

In [36]:
# Filter games with positive reviews.
positive_games = games_clean[games_clean['Sentiment'] > 0]

# Extract Summary text from games with positive reviews
positive_summary_text = positive_games['Summary']

In [37]:
tfidf_positive_summary = TfidfVectorizer(stop_words="english", tokenizer=lemmatize)
tfidf_matrix_positive_summary = tfidf_positive_summary.fit_transform(positive_summary_text)
feature_names_positive_summary = tfidf_positive_summary.get_feature_names_out()

In [38]:
non_empty_feature_indices = [i for i, name in enumerate(feature_names_positive_summary) if name.strip() != '']

# Filter the TF-IDF matrix and feature names
tfidf_matrix_filtered = tfidf_matrix_positive_summary[:, non_empty_feature_indices]
feature_names_filtered = [feature_names_positive_summary[i] for i in non_empty_feature_indices]

In [42]:
# Calculate TF-IDF scores for each word in the positive summary text
tfidf_scores_positive_summary = tfidf_matrix_positive_summary.sum(axis=0)

# Create a DataFrame to display common words and their TF-IDF scores in positive summary text
common_words_positive_summary = pd.DataFrame(tfidf_scores_positive_summary, columns=feature_names_positive_summary).transpose()
common_words_positive_summary.columns = ['TF-IDF Score']

# Filter out empty strings and whitespace characters (optional)
common_words_positive_summary = common_words_positive_summary[common_words_positive_summary.index.str.strip() != '']

# Filter out specific number of letters from words (optional)
common_words_positive_summary = common_words_positive_summary[common_words_positive_summary.index.str.len() > 4]

# Sort the DataFrame by TF-IDF score
common_words_positive_summary = common_words_positive_summary.sort_values(by='TF-IDF Score', ascending=False)

# Display the top common words in positive summary text without blank spaces and single-letter words
top_common_words_positive_summary = common_words_positive_summary.head(20)
print(top_common_words_positive_summary)

            TF-IDF Score
player         29.942295
world          27.922070
character      20.050256
story          19.905167
series         18.386540
adventure      18.202728
feature        18.100277
mario          18.045037
pokmon         16.098179
action         15.969737
experience     15.480059
fight          15.373182
battle         14.580172
gameplay       14.377851
explore        13.835168
enemy          12.820481
combat         12.495122
version        11.881988
super          11.837934
power          11.603026
